In [1]:
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np

import fastf1 as ff1

ff1.Cache.enable_cache("./data/cache")

YEAR = 2022
RACE = "Austria"

session = ff1.get_session(YEAR, RACE, "Race")
session.load(telemetry=True, laps=True, messages=True, weather=False)


core           INFO 	Loading data for Austrian Grand Prix - Race [v2.2.9]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '44', '63', '31', '47', '4', '20', '3', '14', '77', '23', '18', '24', '10', '22', '5', '55', '6', '11']


In [2]:
# lap, position
ALONSO_TEST = [
    (2, 17),
    (24, 9),
    (32, 17),
    (33, 16),
    (40, 13),
    (61, 13),
    (62, 12),
    (67, 11),
    (70, 10),
]


def test_alonso_data(df, strict=False):
    missing = []
    for lap, pos in ALONSO_TEST:
        if df.query(f"LapNumber == {lap} and Position == {pos}").empty:
            print(f"Missing - Lap {lap} for P{pos}")
    if strict:
        print()
        check_lap = df.copy()
        for lap, pos in ALONSO_TEST:
            check_lap = check_lap[
                ~check_lap.filter(["LapNumber", "Position"])
                .isin({"LapNumber": [lap], "Position": [pos]})
                .all(1)
            ]
        if not check_lap.empty:
            print("Unexpected results")
            display(check_lap)


In [88]:
drivers = get_drivers(2022)

df = get_overtakes(session)
df = df.merge(drivers[["DriverNumber", "Code"]], on="DriverNumber", how="left")
df = df.merge(
    drivers[["DriverNumber", "Code"]].rename(
        columns={"DriverNumber": "PassedDriverNumber", "Code": "PassedDriverCode"}
    ),
    on="PassedDriverNumber",
)
df = df[["LapNumber", "PassingStatus", "Code", "Position", "PassedDriverCode"]]
df["LapNumber"] = pd.to_numeric(df["LapNumber"])
df = df.sort_values(by="LapNumber")

with open('./notebooks/test.txt') as f:
    lines = f.readlines()

test_cases = {}
for l in lines:
    parts = l.strip().split(' ')
    lap_number = parts[1]
    driver_name = parts[2]
    passed_name = parts[4]
    position = parts[-1]

    driver = drivers[drivers['LastName'] == driver_name].iloc[0]
    passed = drivers[drivers['LastName'] == passed_name].iloc[0]

    if lap_number not in test_cases:
        test_cases[lap_number] = []
    test_cases[lap_number].append({'code': driver['Code'], 'passed': passed['Code'], 'position': position})

display(test_cases)

{'2': [{'code': 'TSU', 'passed': 'ZHO', 'position': '14'},
  {'code': 'ALO', 'passed': 'VET', 'position': '17'},
  {'code': 'ZHO', 'passed': 'TSU', 'position': '14'}],
 '4': [{'code': 'MSC', 'passed': 'HAM', 'position': '7'}],
 '5': [{'code': 'NOR', 'passed': 'RIC', 'position': '9'}],
 '12': [{'code': 'LEC', 'passed': 'VER', 'position': '1'}],
 '13': [{'code': 'BOT', 'passed': 'GAS', 'position': '16'}],
 '14': [{'code': 'HAM', 'passed': 'MSC', 'position': '5'}],
 '15': [{'code': 'HAM', 'passed': 'MAG', 'position': '4'},
  {'code': 'GAS', 'passed': 'BOT', 'position': '16'},
  {'code': 'RUS', 'passed': 'LAT', 'position': '18'}],
 '16': [{'code': 'VER', 'passed': 'MSC', 'position': '5'}],
 '17': [{'code': 'RUS', 'passed': 'BOT', 'position': '17'}],
 '18': [{'code': 'VER', 'passed': 'HAM', 'position': '3'}],
 '19': [{'code': 'MSC', 'passed': 'RIC', 'position': '13'}],
 '20': [{'code': 'OCO', 'passed': 'VET', 'position': '9'}],
 '21': [{'code': 'RUS', 'passed': 'GAS', 'position': '15'}],
 '

In [128]:
import numpy as np
uut = df[df["PassingStatus"].isnull()]
# uut = uut[uut["PassedDriverCode"].isnull()]
tcs = test_cases.copy()

missing = []

for lap in range(1, 71):
    tc = tcs.get(str(lap))
    t = uut[uut['LapNumber'] == lap].reset_index(drop=True)

    if not tc and t.empty:
        continue
    if tc and t.empty:
        missing.append(f"LAP[{lap}] - missing all in my data set")
        continue
    if not tc and not t.empty:
        missing.append(f"LAP[{lap}] - missing all in target data set")
        continue
    
    for tci in tc:
        code = tci['code']
        passed = tci['passed']
        position = tci['position']
        
        check = t.query(f'Code == "{code}" and PassedDriverCode == "{passed}"')
        if check.empty:
            missing.append(f"LAP[{lap}] - missing {code} on {passed} for {position}")
        else:
            t = t.drop(check.index).reset_index(drop=True)

    for _, sad in t.iterrows():
        code = sad['Code']
        passed = sad['PassedDriverCode']
        position = tci['position']
        missing.append(f"LAP[{lap}] - extra {code} on {passed} for {position}")

            
display(missing)


['LAP[2] - extra VET on ALO for 14',
 'LAP[13] - extra ZHO on TSU for 16',
 'LAP[13] - extra TSU on ZHO for 16',
 'LAP[15] - extra LAT on RUS for 18',
 'LAP[16] - missing all in my data set',
 'LAP[18] - missing all in my data set',
 'LAP[20] - extra VET on PER for 9',
 'LAP[20] - extra LAT on PER for 9',
 'LAP[20] - extra VER on MSC for 9',
 'LAP[20] - extra BOT on LAT for 9',
 'LAP[20] - extra NOR on MSC for 9',
 'LAP[20] - extra GAS on RUS for 9',
 'LAP[20] - extra ALB on GAS for 9',
 'LAP[20] - extra RUS on BOT for 9',
 'LAP[20] - extra MSC on RIC for 9',
 'LAP[20] - extra RIC on ALB for 9',
 'LAP[21] - extra OCO on ALO for 15',
 'LAP[22] - extra MAG on NOR for 7',
 'LAP[22] - extra NOR on MAG for 7',
 'LAP[24] - missing MAG on ZHO for 8',
 'LAP[24] - missing ZHO on ALO for 9',
 'LAP[24] - missing ALO on ZHO for 9',
 'LAP[24] - missing RUS on ALB for 13',
 'LAP[24] - extra NOR on MAG for 13',
 'LAP[24] - extra RUS on RUS for 13',
 'LAP[24] - extra MAG on NOR for 13',
 'LAP[29] - ex

```
Keys:
\ : check
+ : is in my data, but not here
- : in this data, but missing in mine
* : incorrect here
```

\ Lap 2 Tsunoda on Zhou for P 14
\ Lap 2 Alonso on Vettel for P 17
+ Lap 2 Zhou on Tsunoda for P 14
  > This should be filtered out in my data probably. It looks like they battled, but Tsu ultimately came ahead?
  > me ahead tolerance to see

\ Lap 4 Schumacher on Hamilton for P 7

\ Lap 5 Norris on Ricciardo for P 9

Lap 10:
My data here is ok, but by accident? OCO on MAG for P6 is marked `Duplicate` for another OCO on MAG for P6 marked `Retired`

\ Lap 12 Leclerc on Verstappen for P 1

+ Lap 13 Zhou on Tsunoda for P 10
+ Lap 13 Tsunoda on Zhou for P 10
  > Same story as Lap 2
\ Lap 13 Bottas on Gasly for P 16
  > Mine marks this as Pit

\ Lap 14 Hamilton on Schumacher for P 5

\ Lap 15 Hamilton on Magnussen for P 4
* Lap 15 Gasly on Bottas for P 17
  > Mine says P16
\ Lap 15 Russell on Latifi for P 18
+ Lap 15 Latifi on Russel for P 18
  > Same story as Lap 2

Lap 16 Verstappen on Schumacher for P 5

Lap 17 Russell on Bottas for P 17

Lap 18 Verstappen on Hamilton for P 3

Lap 19 Schumacher on Ricciardo for P 13

Lap 20 Ocon on Vettel for P 9

Lap 21 Russell on Gasly for P 15

Lap 22 Ocon on Zhou for P 7

Lap 23 Ocon on Tsunoda for P 6

Lap 24 Magnussen on Zhou for P 8

Lap 24 Alonso on Zhou for P 8

Lap 24 Zhou on Alonso for P 9

Lap 24 Alonso on Zhou for P 9

Lap 24 Magnussen on Alonso for P 9

Lap 24 Norris on Alonso for P 9

Lap 24 Norris on Zhou for P 10

Lap 24 Schumacher on Zhou for P 11

Lap 24 Russell on Albon for P 13

Lap 25 Schumacher on Alonso for P 10

Lap 26 Magnussen on Tsunoda for P 7

Lap 27 Schumacher on Norris for P 8

Lap 29 Russell on Ricciardo for P 10

Lap 30 Hamilton on Ocon for P 4

Lap 30 Vettel on Latifi for P 15

Lap 32 Schumacher on Magnussen for P 6

Lap 32 Tsunoda on Latifi for P 16

Lap 32 Alonso on Latifi for P 17

Lap 32 Zhou on Latifi for P 18

Lap 33 Leclerc on Verstappen for P 1

Lap 33 Alonso on Tsunoda for P 16

Lap 37 Vettel on Bottas for P 13

Lap 37 Zhou on Tsunoda for P 17

Lap 39 Vettel on Gasly for P 12

Lap 40 Alonso on Stroll for P 12

Lap 41 Norris on Magnussen for P 7

Lap 42 Russell on Zhou for P 11

Lap 44 Russell on Stroll for P 8

Lap 44 Russell on Schumacher for P 9

Lap 44 Magnussen on Zhou for P 11

Lap 44 Albon on Vettel for P 15

Lap 45 Stroll on Schumacher for P 8

Lap 45 Schumacher on Stroll for P 8

Lap 46 Norris on Tsunoda for P 12

Lap 47 Schumacher on Stroll for P 8

Lap 47 Magnussen on Stroll for P 9

Lap 50 Zhou on Vettel for P 17

Lap 52 Ocon on Alonso for P 5

Lap 53 Leclerc on Verstappen for P 1

Lap 54 Russell on Alonso for P 6

Lap 54 Zhou on Tsunoda for P 16

Lap 61 Russell on Ocon for P 4

Lap 61 Alonso on Stroll for P 13

Lap 62 Norris on Magnussen for P 7

Lap 62 Alonso on Gasly for P 12

Lap 64 Bottas on Albon for P 10

Lap 64 Stroll on Gasly for P 13

Lap 67 Alonso on Albon for P 11

Lap 67 Zhou on Gasly for P 14

Lap 67 Vettel on Tsunoda for P 16

Lap 70 Alonso on Bottas for P 10